In [1]:
import sys
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from pyDOE import lhs
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_imperfect import Generator, Discriminator, Q_Net
from pid import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda:1')
else:
    device = torch.device('cpu')

## Hyper-parameters

In [3]:
num_epochs = 10000
lambda_val = 0.1
lambda_q = 0.5
tr_frac = 0.4

#architecture for the models
d_hid_dim = 50 
d_num_layer = 2

g_hid_dim = 50
g_num_layer = 4

q_hid_dim = 50
q_num_layer = 3

### Load Data

In [4]:
data = np.loadtxt('../../../datasets/tossing_trajectories.txt')
x = data[:, 0:6]
labels = data[:, 6:]

# training and test split
n_obs = int(tr_frac * x.shape[0])
train_x , train_y = x[:n_obs,:] , labels[:n_obs, :] 
test_x , test_y = x[n_obs:,:] , labels[n_obs:, :]

noise_dim = 2
data_dim = train_x.shape[1]
out_dim = labels.shape[1]

#### Models

In [5]:
D = Discriminator(in_dim = (data_dim + out_dim + 1), out_dim = 1, hid_dim=d_hid_dim, num_layers=d_num_layer).to(device)
G = Generator(in_dim = (noise_dim + data_dim), out_dim = out_dim, hid_dim=g_hid_dim, num_layers=g_num_layer).to(device)
Q = Q_Net(in_dim=(out_dim + data_dim), out_dim=noise_dim, hid_dim=q_hid_dim, num_layers=q_num_layer).to(device)

In [6]:
PID = Tossing_PID(train_x, train_y, test_x, test_y, G, D, Q, device, num_epochs, lambda_val, noise_dim)

### Training

In [7]:
PID.train()

[Epoch 0/10000] [MSE loss: 3.974961] [G loss: 1.534164] [D loss: 8.440324] [Q loss: 4.142524] [Phy loss: 444.450264] [Adv G loss: -0.521032]
[Epoch 100/10000] [MSE loss: 0.167551] [G loss: 13.524945] [D loss: 3.584815] [Q loss: 0.849222] [Phy loss: 27.464101] [Adv G loss: 12.979776]
[Epoch 200/10000] [MSE loss: 0.306981] [G loss: 21.852914] [D loss: 3.001302] [Q loss: 0.468556] [Phy loss: 35.642198] [Adv G loss: 20.916393]
[Epoch 300/10000] [MSE loss: 0.099735] [G loss: 16.830311] [D loss: 5.433719] [Q loss: 0.792401] [Phy loss: 10.138143] [Adv G loss: 15.639487]
[Epoch 400/10000] [MSE loss: 0.104476] [G loss: 15.371575] [D loss: 6.396858] [Q loss: 0.423573] [Phy loss: 9.746002] [Adv G loss: 14.622990]
[Epoch 500/10000] [MSE loss: 0.191786] [G loss: 16.322589] [D loss: 5.075573] [Q loss: 0.553257] [Phy loss: 20.125237] [Adv G loss: 16.025180]
[Epoch 600/10000] [MSE loss: 0.105447] [G loss: 15.116018] [D loss: 5.393000] [Q loss: 0.442200] [Phy loss: 10.813882] [Adv G loss: 14.306361]
[E

In [8]:
nsamples = 500
train_predictions_list = []
for run in range(nsamples):
    G_train_noise = PID.sample_noise(PID.train_x.shape[0], PID.noise_dim).to(device)
    train_predictions = PID.G.forward(torch.cat([PID.train_x, G_train_noise], dim=1)).detach().cpu().numpy()
    train_predictions = (train_predictions * PID.Ystd) + PID.Ymean
    train_predictions_list.append(train_predictions)

train_predictions = np.mean(train_predictions_list, axis=0)
train_predictions_dev = np.var(train_predictions_list, axis=0)
train_predictions = torch.Tensor(train_predictions).float().to(device)

std_y_t = torch.Tensor(PID.Ystd).float().to(device)
mean_y_t = torch.Tensor(PID.Ymean).float().to(device)
std_x_t = torch.Tensor(PID.Xstd).float().to(device)
mean_x_t = torch.Tensor(PID.Xmean).float().to(device)

train_y = (PID.train_y * std_y_t) + mean_y_t
train_x = (PID.train_x * std_x_t) + mean_x_t

mse_train = ((train_predictions - train_y)**2).mean()
phy_train = torch.mean(PID.physics_loss(train_x, train_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Training Set : ", mse_train.detach().cpu().numpy())
print("Physical Consistency: ", phy_train)

MSE on Training Set :  0.07892624
Physical Consistency:  0.5669054


In [9]:
test_predictions_list = []
for run in range(nsamples):
    G_test_noise = PID.sample_noise(PID.test_x.shape[0], PID.noise_dim).to(device)
    test_predictions = PID.G.forward(torch.cat([PID.test_x, G_test_noise], dim=1)).detach().cpu().numpy()
    test_predictions = (test_predictions * PID.Ystd) + PID.Ymean
    test_predictions_list.append(test_predictions)

test_predictions = np.mean(test_predictions_list, axis=0)
test_predictions_dev = np.var(test_predictions_list, axis=0)
test_predictions = torch.Tensor(test_predictions).float().to(device)

test_y = (PID.test_y * std_y_t) + mean_y_t
test_x = (PID.test_x * std_x_t) + mean_x_t

mse_test = ((test_predictions - test_y)**2).mean()
phy_test = torch.mean(PID.physics_loss(test_x, test_predictions, [0, 1], [0, 1])).detach().cpu().numpy()

print("MSE on Test Set : ", mse_test.detach().cpu().numpy())
print("Physical Consistency: ", phy_test)

MSE on Test Set :  0.08371187
Physical Consistency:  0.6453264
